# SciKitLearn.CrossValidation

Тут будем разбивать разово и просмотрим наиболее часто используемые методологии

In [1]:
from sklearn import datasets, cross_validation
import numpy as np

# Разовое разбиение данных на обучение и тест с помощью train_test_split

train_test_split - функция разово разделяет выборку 
Это полезно если мы хотим оценить качество нашей модели в одной точке если же нужна более строгая оценка надо использовать CrossValidation

In [2]:
iris=datasets.load_iris()
iris

{'DESCR': 'Iris Plants Database\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)\n    ============== ==== ==== ======= ===== ====================\n\n

In [3]:
train_data, test_data, train_label, test_label = cross_validation.train_test_split(iris.data
                                                                                   , iris.target
                                                                                   , test_size=0.3)

In [24]:
##проверим процент покрытия
float(len(test_label))/float(len(iris.data))
print "Размер обучающе выборки {} \nРазмер тестовой выборки {}".format(len(train_data), len(test_data))
print "Обучающая выборка :\n", train_data[:5]
print "Тестовая выборка :\n",  test_data[:5]

##Обратим внимание чтобы и в обучении и в тесте присутствуют объекты всех классов
print "Метки классов на Обучающая выборка :\n", train_label
print "Метки классов на Тестовая выборка :\n",  test_label

Размер обучающе выборки 105 
Размер тестовой выборки 45
Обучающая выборка :
[[ 4.8  3.4  1.6  0.2]
 [ 4.4  3.2  1.3  0.2]
 [ 5.   3.   1.6  0.2]
 [ 6.   3.4  4.5  1.6]
 [ 5.   3.2  1.2  0.2]]
Тестовая выборка :
[[ 7.9  3.8  6.4  2. ]
 [ 6.7  3.3  5.7  2.1]
 [ 6.1  2.8  4.   1.3]
 [ 5.8  2.7  3.9  1.2]
 [ 6.7  3.1  4.4  1.4]]
Метки классов на Обучающая выборка :
[0 0 0 1 0 1 1 1 0 1 2 1 2 2 1 0 0 0 2 2 2 2 1 2 2 1 2 2 2 1 1 0 0 2 0 2 1
 1 0 1 0 1 1 2 0 1 2 0 2 0 2 1 0 1 1 0 1 0 2 1 0 0 0 1 1 1 0 0 1 2 1 2 1 0
 1 1 2 2 0 0 2 2 1 1 1 0 1 2 0 1 2 2 1 0 0 1 0 0 2 2 0 1 0 2 0]
Метки классов на Тестовая выборка :
[2 2 1 1 1 1 2 0 1 1 2 0 0 0 1 1 1 0 2 2 2 2 0 2 1 2 0 2 2 2 0 2 2 0 2 0 0
 0 0 1 2 2 1 2 0]


# Стратегии проведения CrossValidation

CrossValidation - более строгая оценка качества модели. Рассмотрим несколько стратегий ее проведения:
1. KFold - при этом вся выборка разбивается на K групп. При этом каждая группа 1 раз участвует в тестировании и К-1 раз в обучении.

In [9]:
#Пример разбития где последовательность чисел с 0 по 9 разбивается на 5 фолдов
# как видно каждый элемент участвует 1 раз в тестировании и 4 раза в обучении
for train_indices, test_indicies in cross_validation.KFold(10, n_folds=5):
    print train_indices, test_indicies

[2 3 4 5 6 7 8 9] [0 1]
[0 1 4 5 6 7 8 9] [2 3]
[0 1 2 3 6 7 8 9] [4 5]
[0 1 2 3 4 5 8 9] [6 7]
[0 1 2 3 4 5 6 7] [8 9]


In [17]:
# параметр shuffle перемешивает элементы в выборке
# random_state фиксирует распределение что бы его можно было потом воспроизвести
for train_indices, test_indicies in cross_validation.KFold(10, n_folds=5, shuffle=True, random_state=1):
    print train_indices, test_indicies

[0 1 3 4 5 6 7 8] [2 9]
[0 1 2 3 5 7 8 9] [4 6]
[1 2 4 5 6 7 8 9] [0 3]
[0 2 3 4 5 6 8 9] [1 7]
[0 1 2 3 4 6 7 9] [5 8]


2. "Stratified KFold" - стратегия оставляющая соотношения классов в обучающей и тестовой подвыборках.
При этом передавать нужно не только количество объектов но и метки классов на объектах:

In [21]:
# создаем набор меток классов:(К примеру : первые 2 бутут =0 последние 8 =1)
target =np.array([0]*2+[1]*8)
print target
#pay attention distribution in training representation as well as basic class 20%
for train_indices, test_indicies in cross_validation.StratifiedKFold(target, n_folds=2, shuffle=True, random_state=1):
    print train_indices, test_indicies

[0 0 1 1 1 1 1 1 1 1]
[1 2 3 5 6] [0 4 7 8 9]
[0 4 7 8 9] [1 2 3 5 6]


3. "Shuffle Split" - позволяет строить случайные перестановки т.о. мы можем получить очень много выборок. Мы специфицируем размер обучающей выборки и у нас нет ограничения на то сколько раз объект появится в обучении или в тесте

In [22]:
for train_indices, test_indicies in cross_validation.ShuffleSplit(10, n_iter=10, test_size=0.2):
    print train_indices, test_indicies

[8 4 6 1 5 7 0 3] [2 9]
[6 5 9 2 7 0 3 8] [4 1]
[0 6 7 4 9 2 3 5] [8 1]
[5 6 4 1 0 3 9 7] [2 8]
[2 8 4 5 3 7 9 0] [6 1]
[0 8 1 5 2 9 6 7] [4 3]
[2 3 1 7 5 6 8 0] [4 9]
[3 2 7 8 4 9 6 0] [1 5]
[8 2 0 4 9 7 1 6] [5 3]
[3 2 5 7 8 9 4 6] [1 0]


4. "Stratified Shuffle Split" - произвольно распределяет с сохранением исходного соотношения классов

In [25]:
# создаем набор меток классов:(К примеру : первые 2 бутут =0 последние 8 =1)
target =np.array([0, 1]*5)
print target
for train_indices, test_indicies in cross_validation.StratifiedShuffleSplit(target, n_iter=4, test_size=0.2):
    print train_indices, test_indicies

[0 1 0 1 0 1 0 1 0 1]
[6 1 4 8 7 3 2 9] [0 5]
[6 5 1 0 8 3 7 2] [9 4]
[3 9 4 2 0 1 8 5] [6 7]
[0 5 4 7 3 8 9 2] [1 6]


5. "Leave One Out" - позволяет оставить объект в тесте лишь один раз. 
Т.о. тестовая выборка состоит из одного объекта. Каждый объект лишь один раз будет присутствовать в тесте.
Хорошая стратегия если мы имеем небольшую выборку данных.

In [26]:
for train_indices, test_indicies in cross_validation.LeaveOneOut(10):
    print train_indices, test_indicies

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]


Больше стратегий проведения кросс-валидации доступно здесь: http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators